In [1]:
import pandas as pd

Read all data from csv file.

In [3]:
word_association = pd.read_csv('EDA_with_Pandas.csv')

In [5]:
type_association = word_association['Тип асоціації'].value_counts()
type_association

Тип асоціації
слово контексту    780
синонім            219
антонім             33
Name: count, dtype: int64